# Modulbeschreibung

<table style="font-size: 1em; margin-left: 0; margin-top: 2em; text-align: left">
    <tr>
        <th style="border-style: none">Name</th>
        <td style="border-style: none">TODO</td>
    </tr>
    <tr>
        <th style="border-style: none">Zielgruppe</th>
        <td style="border-style: none">
            Studierende im 4. Semester der B.Sc.-Studiengänge (Wirtschafts-)Informatik
        </td>
    </tr>
    <tr>
        <th rowspan="2" style="border-style: none">Voraussetzungen</th>
        <td style="border-style: none">grundlegende Datenbank-Kenntnisse (bspw. aus der Vorlesung <q>Datenbanksysteme</q>)</td>
    </tr>
    <tr>
        <td style="border-style: none">grundlegende Kenntnisse der objektorientierten Programmierung (bspw. aus TODO)</td>
    </tr>
    <tr>
        <th rowspan="2" style="border-style: none">Ziele</th>
        <td style="border-style: none">TODO</td>
    </tr>
    <tr>
        <td style="border-style: none">TODO</td>
    </tr>
    <tr>
        <th rowspan="3" style="border-style: none">Inhalt</th>
        <td style="border-style: none">TODO</td>
    </tr>
    <tr>
        <td style="border-style: none">TODO</td>
    </tr>
    <tr>
        <td style="border-style: none">TODO</td>
    </tr>
</table>

# JSON-Unterstützung in klassischen und NoSQL-Datenbanken anhand eines praktischen Beispiels

<table style="font-size: 1em; margin-left: 0; margin-top: 2em; text-align: left">
    <tr>
        <th style="border-style: none; padding-left: 0">Modul</th>
        <td style="border-style: none">TODO</td>
    </tr>
    <tr>
        <th style="border-style: none; padding-left: 0">Dozent</th>
        <td style="border-style: none">Christoph Gietl</td>
    </tr>
    <tr>
        <th style="border-style: none; padding-left: 0">Datum</th>
        <td style="border-style: none">17. Mai 2022</td>
    </tr>
    <tr>
        <th style="border-style: none; padding-left: 0">Notebook und Ressourcen</th>
        <td style="border-style: none">
            <a href="https://github.com/christophgietl/json-in-datenbanken">github.com/christophgietl/json-in-datenbanken</a>
        </td>
    </tr>
</table>

## JSON - Das Quiz

### Wofür steht die Abkürzung *JSON*?

1. Java Standard Output Network
2. June September October November
3. JavaScript Object Notation
4. Just Some Open Network

#### Lösung

Die richtige Antwort lautet *JavaScript Object Notation*.
Trotz des JavaScript im Namen wird JSON heute von einer Vielzahl an Programmiersprachen unterstützt.

### Welcher der folgenden Texte ist ein valides JSON-Dokument?

1. `2`
2. `{"id": 5, "name": "Franziska"}`
3. `[1986, True, "foo"]`
4. `"Ali"`

#### Lösung

Die Antworten 1, 2 und 4 sind valide JSON-Dokumente.
In Antwort 3 muss `True` durch `true` ersetzt werden, um ein valides JSON-Dokument zu erhalten.

### Welcher der folgenden Begriffe bezeichnet einen Datentyp in JSON?

1. List
2. Object
3. String
4. Float

#### Lösung

Die Antworten 2 und 3 sind JSON-Datentypen.
Listen heißen in JSON *Arrays*.
Den Datentyp Float gibt es in JSON nicht. Stattdessen gibt es den Datentyp *Number*, der Ganzzahlen und Dezimalzahlen enthalten kann.

### Welche Methode der Python-Standard-Library (PSL) kann ein JSON-Dokument deserialisieren, das als `str` vorliegt?

1. `json.loads`
2. `json.deserialize`
3. `deserialize.json`
4. `json.dump`

#### Lösung

Antwort 2 ist richtig.
Die PSL enthält keine Methoden namens `json.deserialize` oder `deserialize.json`.
Die Methode `json.dump` serialisiert Python-Objekte und speichert sie in einem "file-like object" (z.B. eine Datei).

### Welchers Ergebniss liefert die PSL bei der Deserialisierung des JSON-Dokuments `{"name": "Eva", "misc": [5, "dog"]}`?

1. eine Fehlermeldung
2. ein `dict` mit zwei Schlüsseln vom Typ `str` und zwei Werten vom Typ `str`
3. ein `tuple` mit vier Elementen
4. ein `dict` mit zwei Schlüsseln vom Typ `str`, einem Wert vom Typ `str` und einem Wert vom Typ `list`

#### Lösung

Antwort 4 ist richtig.
Die JSON-Typen werden [gemäß einer Tabelle](https://docs.python.org/3.10/library/json.html#json.JSONDecoder) in entsprechende Python-Klassen konvertiert. Dabei werden JSON-Objekte in `dict`s und JSON-Arrays in `list`s konvertiert.

Im Folgenden verzichten wir auf eine begriffliche Trennung zwischen JSON-Dokumenten und als JSON-Dokument serialisierbaren Python-Objekten:

In [1]:
json_document = '{"name": "Eva", "misc": [5, "dog"]}'
type(json_document)

str

In [2]:
python_object = {"name": "Eva", "misc": [5, "dog"]}
type(python_object)

dict

Stattdessen bezeichnen wir beide als JSON-Dokumente.

## Praxisbeispiel: Nutzer:innenprofile in einer Stellenbörse

Eine Stellenbörse will ihren Nutzer:innen ermöglichen, ein Profil anzulegen.
Dieses Nutzer:innenprofil kann wie folgt aussehen:

<img src="images/user_profile.png" width="300" border="1"/>

**Quelle:** M. Kleppmann: *Designing Data-Intensive Applications*

Das Profil enthält einige Attribute des Typs String:
- "Bill" (Vorname)
- "Gates" (Nachname)
- "Co-chair of the ... Avid traveler. Active blogger." (Zusammenfassung)
- "Greater Seattle Area" (Region)
- "Philanthropy" (Branche)

Darüberhinaus enthält das Profil einige komplexe Attribute.
Zum Beispiel hat das Attribut *Education* als Wert dieses Attributs eine Liste von Stationen.
Jede dieser Station besteht aus dem Namen der Bildungseinrichtung und weiteren optionalen Informationen wie dem Ort oder Anfang und Ende des Besuchs.

Das Profil kann in einem JSON-Dokument zusammengefasst werden:

In [3]:
user_251 = {
    "_id": 251,
    "first_name": "Bill",
    "last_name": "Gates",
    "summary": "Co-chair of the Bill & Melinda Gates... Active blogger.",
    "region": "Greater Seattle Area",
    "industry": "Philanthropy",
    "positions": [
        {"job_title": "Co-chair", "organization": "Bill & Melinda Gates Foundation"},
        {"job_title": "Co-founder, Chairman", "organization": "Microsoft"}
    ],
    "education": [
        {"school_name": "Harvard University", "start": 1973, "end": 1975},
        {"school_name": "Lakeside School, Seattle"}
    ],
    "contact_info": {
        "blog": "https://www.gatesnotes.com/",
        "twitter": "https://twitter.com/BillGates"
    }
}

Die Nutzer:innen der Stellenbörse erwarten beim Aufruf einer Profilseite eine schnelle Antwort des Webservers.
Daher möchte die Stellenbörse alle Profilinformationen eines Nutzers gebündelt in einem Dokument speichern.

Als Technologiestack stehen der Stellenbörse die MongoDB-Instanz `demo-mongo` und die PostgreSQL-Instanz `demo-postgres` zur Verfügung:

In [4]:
import pathlib
tech_stack = pathlib.Path("docker-compose.yml")
print(tech_stack.read_text())

version: "3.0"

services:
  demo-mongo:
    image: mongo:4.4
  demo-postgres:
    environment:
      POSTGRES_HOST_AUTH_METHOD: trust
    image: postgres:14.2
  jupyter:
    depends_on:
      - demo-mongo
      - demo-postgres
    image: jupyter/base-notebook:notebook-6.4.11
    ports:
      - "8888:8888"
    volumes:
      - .:/home/jovyan/work



### Umsetzung in einem dokumentenorientierten Datenbankmanagementsystem

#### MongoDB als dokumentenorientiertes Datenbankmanagementsystem

MongoDB
- ist ein dokumentenorientiertes No-SQL-Datenbankmanagementsystem,
- wird von *MongoDB, Inc.* entwickelt,
- steht unter der Server-Side-Public-License (eine Modifikation der zuvor verwendeten GNU Affero General Public License),
- verwendet [MongoDB Extended JSON](https://www.mongodb.com/docs/v4.4/reference/mongodb-extended-json/) als logisches Datenmodell und
- verwendet [BSON](https://www.mongodb.com/docs/v4.4/reference/glossary/) ("binary JSON") als physisches Datenmodell.

Der Name geht auf das englische Wort *hu**mongo**us* (gigantisch) zurück.

#### Datenbankverbindungen verwalten

Das Python-Paket `pymongo` steht unter der Apache Software License 2.0.
Es erlaubt die Nutzung von MongoDB in Python.
Die wichtigste Klasse des Pakets heißt `pymongo.MongoClient`.
Sie ermöglicht den Zugriff auf MongoDB-Instanzen.

Wir installieren das Paket `pymongo` und erstellen ein `pymongo.MongoClient`-Objekt für den Zugriff auf die MongoDb-Instanz `demo-mongo`:

In [5]:
!pip install pymongo~=4.1.1

In [6]:
import pymongo
demo_mongo_client = pymongo.MongoClient("mongodb://demo-mongo")

#### Dokumente organisieren

Jede MongoDB-Instanz verwaltet ihre Dokumente in einer dreistufigen Hierarchie:
1. MongoDB-Instanzen enthalten Datenbanken.
2. Datenbanken enthalten Collections.
3. Collections enthalten Dokumente.

**Achtung:** MongoDB akzeptiert nur JSON-Dokumente vom Typ JSON-Objekt als (MongoDB-)Dokumente. Diese JSON-Objekte können als Werte beliebige JSON-Dokumente enthalten (bspw. JSON-Arrays, JSON-Objekte oder JSON-Strings).

Datenbanken und Collections müssen nicht explizit angelegt werden.
Sie werden automatisch angelegt, wenn zum ersten Mal auf sie zugegriffen wird.

Wir verwalten die Dokumente unserer Stellenbörse in der Datenbank `employment`:

In [7]:
employment = demo_mongo_client["employment"]
type(employment)

pymongo.database.Database

Die Profile der Nutzer:innen speichern wir in der Collection `users`:

In [8]:
users = employment["users"]
type(users)

pymongo.collection.Collection

Zu Beginn unseres Beispiels stellen wir sicher, dass die Collection `users` leer ist: 

In [9]:
users.drop()

#### Dokumente einfügen

Die Methode `insert_one` der Klasse `pymongo.collection.Collection` fügt ein JSON-Objekt zur Collection hinzu. Falls das JSON-Objekt ein Attribut mit dem Namen `_id` enthält, verwendet MongoDB dieses Attribut als Primärschlüssel. Falls nicht, erzeugt MongoDB einen Primärschlüssel und fügt ihn als Attribut mit dem Namen `_id` hinzu.

Wir fügen ein JSON-Objekt zur Collection `users` hinzu:

In [10]:
users.insert_one(user_251)

#### Dokumenten abfragen

Die Methode `find` der Klasse `pymongo.collection.Collection` ermöglicht das Abfragen von Dokumenten.

##### Alle Dokumente abfragen

Die Methode `find` liefert einen Cursor als Rückgabewert:

In [11]:
documents = users.find({})
documents

Wir konstruieren eine Liste, um alle Ergebnisse der Abfrage zu erhalten:

In [12]:
list(documents)

[{'_id': 251,
  'first_name': 'Bill',
  'last_name': 'Gates',
  'summary': 'Co-chair of the Bill & Melinda Gates... Active blogger.',
  'region': 'Greater Seattle Area',
  'industry': 'Philanthropy',
  'positions': [{'job_title': 'Co-chair',
    'organization': 'Bill & Melinda Gates Foundation'},
   {'job_title': 'Co-founder, Chairman', 'organization': 'Microsoft'}],
  'education': [{'school_name': 'Harvard University',
    'start': 1973,
    'end': 1975},
   {'school_name': 'Lakeside School, Seattle'}],
  'contact_info': {'blog': 'https://www.gatesnotes.com/',
   'twitter': 'https://twitter.com/BillGates'}}]

##### Felder einschränken

Die Methode `find` nimmt ein zweites Argument entgegen.
Mit diesem Argument weisen wir die MongoDB-Instanz an, nur bestimmte Felder der Dokumente zurückzugeben.

In [13]:
documents = users.find({}, {"first_name": 1, "positions.organization": 1})
list(documents)

[{'_id': 251,
  'first_name': 'Bill',
  'positions': [{'organization': 'Bill & Melinda Gates Foundation'},
   {'organization': 'Microsoft'}]}]

Mit dem Punkt in `positions.organization` konnten wir auf das Attribut `organization` der Objekte im Attribut `positions` zugreifen.
Da es sich bei `positions` um ein Array von Objekten handelt, haben wir auch ein Array von Objekten erhalten (mit dem jeweils einizigen Attribut `organization`).

Zusätzlich zu den von uns spezifizierten Attributen `first_name` und `positions.organization` haben wir noch den Primärschlüssel `_id` erhalten.
Wenn wir den Primärschlüssel nicht erhalten wollen, müssen wir ihn explizit ausschließen:

In [14]:
names_only = {"_id": 0, "first_name": 1, "last_name": 1}
documents = users.find({}, names_only)
list(documents)

[{'first_name': 'Bill', 'last_name': 'Gates'}]

##### Dokumente einschränken

Neben den Feldern können wir auch einschränken, welche Dokumente der Collection die Methode `find` zurückgeben soll.

Wenn wir nach einem bestimmten Primärschlüssel suchen, erhalten wir maximal ein Dokument:

In [15]:
documents = users.find({"_id": 251}, names_only)
list(documents)

[{'first_name': 'Bill', 'last_name': 'Gates'}]

In [16]:
documents = users.find({"_id": 12}, names_only)
list(documents)

[]

Neben dem Primärschlüssel können wir auch nach anderen Attributen suchen:

In [17]:
documents = users.find({"last_name": "Gates"}, names_only)
list(documents)

[{'first_name': 'Bill', 'last_name': 'Gates'}]

Mit dem Punkt können wir auch nach Attributen von Unterobjekten suchen:

In [18]:
documents = users.find(
    {"contact_info.twitter": "https://twitter.com/BillGates"}
)
list(documents)

[{'_id': 251,
  'first_name': 'Bill',
  'last_name': 'Gates',
  'summary': 'Co-chair of the Bill & Melinda Gates... Active blogger.',
  'region': 'Greater Seattle Area',
  'industry': 'Philanthropy',
  'positions': [{'job_title': 'Co-chair',
    'organization': 'Bill & Melinda Gates Foundation'},
   {'job_title': 'Co-founder, Chairman', 'organization': 'Microsoft'}],
  'education': [{'school_name': 'Harvard University',
    'start': 1973,
    'end': 1975},
   {'school_name': 'Lakeside School, Seattle'}],
  'contact_info': {'blog': 'https://www.gatesnotes.com/',
   'twitter': 'https://twitter.com/BillGates'}}]

Wenn das Attribut links des Punkts den Typ Array hat, genügt es, wenn eines der Elemente ein Objekt mit dem gewünschten Attribut ist:

In [19]:
documents = users.find(
    {"positions.organization": "Microsoft"},
    {"_id": 0, "first_name": 1, "last_name": 1, "positions": 1}
)
list(documents)

[{'first_name': 'Bill',
  'last_name': 'Gates',
  'positions': [{'job_title': 'Co-chair',
    'organization': 'Bill & Melinda Gates Foundation'},
   {'job_title': 'Co-founder, Chairman', 'organization': 'Microsoft'}]}]

#### Dokumente verändern

Die Methode `update_many` der Klasse `pymongo.collection.Collection` ermöglicht das Verändern von Dokumenten:

In [20]:
users.update_many({"_id": 251}, {"$set": {"first_name": "William"}})
documents = users.find({"_id": 251}, names_only)
list(documents)

[{'first_name': 'William', 'last_name': 'Gates'}]

#### Dokumente löschen

Die Methode `delete_many` der Klasse `pymongo.collection.Collection` ermöglicht das Löschen von Dokumenten:

In [21]:
users.delete_many({"last_name": "Gates"})
documents = users.find({"_id": 251}, names_only)
list(documents)

[]

#### Referenzen zwischen Dokumenten

Mit wachsender Nutzer:innenzahl der Stellenbörse wollen wir unsere Nutzer:innen-Dokumente normalisieren.
Das heißt, wir wollen
1. einzelne Informationen der Nutzer:innen in eigene Collections auslagern (bspw. die Branche (`industry`)) und
2. das Dokument der jeweiligen Branche im Dokument des:der Nutzer:in referenzieren.

Dafür legen wir die Collection `industries` an:

In [22]:
industries = employment["industries"]
industries.drop()
industries.insert_many([
    {
        "_id": 43,
        "name": "Financial Services",
        "description": "Banking, etc."
    },
    {"_id": 48, "name": "Construction"},
    {"_id": 131, "name": "Philanthropy"}
])

Außerdem legen wir die Collection `regions` an:

In [23]:
regions = employment["regions"]
regions.drop()
regions.insert_many([
    {"_id": "us:7", "name": "Greater Boston Area"},
    {
        "_id": "us:91",
        "name": "Greater Seattle Area",
        "state": "Washington"
    }
])

Wir fügen nun einen Nutzer hinzu, dessen Attribute `industry` und `region` auf jeweils ein Dokument der entsprechenden Collection verweisen:

In [24]:
import bson.dbref
users.insert_one({
    "_id": 251,
    "first_name": "Bill",
    "last_name": "Gates",
    "summary": "Co-chair of the Bill & Melinda Gates... Active blogger.",
    "region": bson.dbref.DBRef(collection=regions.name, id="us:91"),
    "industry": bson.dbref.DBRef(collection=industries.name, id="131"),
    "positions": [
        {"job_title": "Co-chair", "organization": "Bill & Melinda Gates Foundation"},
        {"job_title": "Co-founder, Chairman", "organization": "Microsoft"}
    ],
    "education": [
        {"school_name": "Harvard University", "start": 1973, "end": 1975},
        {"school_name": "Lakeside School, Seattle"}
    ],
    "contact_info": {
        "blog": "https://www.gatesnotes.com/",
        "twitter": "https://twitter.com/BillGates"
    }
})

Eine Abfrage dieses Nutzers liefert keinen menschenlesbaren Namen der Region.
Stattdessen wird ein Fremdschlüssel auf die Collection `regions` geliefert:

In [25]:
user = users.find_one(
    {"_id": 251},
    {"_id": 0, "last_name": 1, "region": 1}
)
user

{'last_name': 'Gates', 'region': DBRef('regions', 'us:91')}

Dieser Fremdschlüssel muss von der Anwendung aufgelöst werden.
Die Klasse `pymongo.database.Database` stellt dafür die Methode `dereference` bereit:

In [26]:
employment.dereference(user["region"])

{'_id': 'us:91', 'name': 'Greater Seattle Area', 'state': 'Washington'}

Wir haben also zwei Datenbank-Abfragen benötigt, um den Namen der Region des Nutzers mit dem Primärschlüssel `251` zu finden:
1. Finde das Dokument mit dem Primärschlüssel `251` in der Collection `users`.
2. Finde das Dokument mit dem Primärschlüssel `us:91` in der Collection `regions`.

Relationale Datenbankmanagementsysteme erlauben die Verbindung zweier Tabellen in einer Abfrage mithilfe des `join`-Statements.
Eine vergleichbare Verbindung zweier Collections in einer Abfrage wird von MongoDB nicht unterstützt.

### Umsetzung in einem relationalen Datenbankmanagementsystem

#### PostgreSQL als relationales Datenbankmanagementsystem

PostgreSQL
- ist ein relationales-Datenbankmanagementsystem,
- steht unter der PostgreSQL-License (eine Free-and-Open-Source-Lizenz),
- verwendet das relationale Datenmodell als logisches Datenmodell.

Die JSON-Unterstützung wurde in den letzten Jahren schrittweise ausgebaut.
Für die Verwaltung von JSON-Dokumenten wurden zwei Typen hinzugefügt:
1. Der Typ `json` speichert JSON-Dokumente als Texte. Im Gegensatz zum Typ `text` wird vor dem Speichern eine JSON-Validierung durchgeführt.
2. Der Typ `jsonb` speichert JSON-Dokumente in einem speziellen Binärformat. Die Umwandlung verursacht zusätzlichen Overhead beim Schreiben. Im Gegenzug werden Abfragen in der Regel beschleunigt.

#### Datenbankverbindungen verwalten

Das Python-Paket `psycopg` steht unter der GNU LGPL 3.0.
Es erlaubt die Nutzung von PostgreSQL in Python.
Die wichtigste Klasse des Pakets heißt `psycopg.Connection`.
Sie ermöglicht den Zugriff auf PostgreSQL-Instanzen.

Wir installieren das Paket `psycopg` und erstellen ein `psycopg.Connection`-Objekt für den Zugriff auf die PostgreSQL-Instanz `demo-postgres`:

In [27]:
!pip install psycopg[binary]~=3.0.13

In [28]:
import psycopg
conn = psycopg.connect("postgresql://postgres@demo-postgres/postgres", autocommit=True)
type(conn)

psycopg.Connection

Außerdem erstellen wir ein Objekt der Klasse `psycopg.Cursor`:

In [29]:
cur = conn.cursor()
type(cur)

psycopg.Cursor

Die Klasse `psycopg.Cursor` erlaubt
1. das Ausführen von SQL-Queries mithilfe der Methode `execute` und
2. das Abrufen der Resultate der letzten SQL-Query mithilfe der Methode `fetchall`.

#### Dokumente organisieren

Wir erstellen die Tabelle `users` mit der Spalte `doc` vom Typ `jsonb`:

In [30]:
cur.execute("drop table if exists users")
cur.execute("create table users(doc jsonb)")

<psycopg.Cursor [COMMAND_OK] [IDLE] (host=demo-postgres database=postgres) at 0x7f6c0034e1a0>

#### Dokumente einfügen

PostgreSQL erlaubt das Einfügen von Texten in Spalten vom Typ `jsonb`, sofern es sich bei diesen Texten um valide JSON-Dokumente handelt:

```sql
insert into users (doc) values ('{ "first_name" : "Bill"}');
```

PostgreSQL wandelt diese Texte automatisch in den Typ `jsonb` um.

Für die Serialisierung von Python-Objekten als JSON-Dokumente stellt `psycopg` die Wrapper-Klasse `psycopg.types.json.Jsonb` bereit:

In [31]:
import psycopg.types.json
cur.execute(
    "insert into users (doc) values (%s)",
    (psycopg.types.json.Jsonb(user_251),)
)

<psycopg.Cursor [COMMAND_OK] [IDLE] (host=demo-postgres database=postgres) at 0x7f6c0034e1a0>

#### Dokumente abfragen

##### Alle Dokumente abfragen

Spalten vom Typ `jsonb` können mithilfe eines `select`-Statements abefragt werden (genau wie andere Spalten):

In [32]:
cur.execute("select doc from users")
users = cur.fetchall()
users

[({'_id': 251,
   'region': 'Greater Seattle Area',
   'summary': 'Co-chair of the Bill & Melinda Gates... Active blogger.',
   'industry': 'Philanthropy',
   'education': [{'end': 1975,
     'start': 1973,
     'school_name': 'Harvard University'},
    {'school_name': 'Lakeside School, Seattle'}],
   'last_name': 'Gates',
   'positions': [{'job_title': 'Co-chair',
     'organization': 'Bill & Melinda Gates Foundation'},
    {'job_title': 'Co-founder, Chairman', 'organization': 'Microsoft'}],
   'first_name': 'Bill',
   'contact_info': {'blog': 'https://www.gatesnotes.com/',
    'twitter': 'https://twitter.com/BillGates'}},)]

Der Output der Methode `fetchall` ist wie üblich eine Liste von Tupeln:

In [33]:
type(users)

list

In [34]:
type(users[0])

tuple

Mithilfe der Zeilen- und der Spaltennummer können wir auf einzelne JSON-Dokumente zugreifen:

In [35]:
users[0][0]

{'_id': 251,
 'region': 'Greater Seattle Area',
 'summary': 'Co-chair of the Bill & Melinda Gates... Active blogger.',
 'industry': 'Philanthropy',
 'education': [{'end': 1975,
   'start': 1973,
   'school_name': 'Harvard University'},
  {'school_name': 'Lakeside School, Seattle'}],
 'last_name': 'Gates',
 'positions': [{'job_title': 'Co-chair',
   'organization': 'Bill & Melinda Gates Foundation'},
  {'job_title': 'Co-founder, Chairman', 'organization': 'Microsoft'}],
 'first_name': 'Bill',
 'contact_info': {'blog': 'https://www.gatesnotes.com/',
  'twitter': 'https://twitter.com/BillGates'}}

Diese JSON-Dokumente werden von `fetchall` automatisch in ein Objekt einer geeigneten Python-Klasse umgewandelt:

In [36]:
type(users[0][0])

dict

Mit diesen JSON-Objekten können wir daher arbeiten wie unter Python üblich:

In [37]:
users[0][0]["education"]

[{'end': 1975, 'start': 1973, 'school_name': 'Harvard University'},
 {'school_name': 'Lakeside School, Seattle'}]

##### Felder einschränken

Statt die gesamten User-Dokumente abzufragen, können wir auch auf einzelne Felder der Dokumente zugreifen:

In [38]:
cur.execute("""
    select
        doc['last_name'],
        doc['education'][0]
    from users
""")

<psycopg.Cursor [TUPLES_OK] [IDLE] (host=demo-postgres database=postgres) at 0x7f6c0034e1a0>

Die beiden Spalten `doc['last_name']` und `doc['education'][0]` der resultierenden Tabelle haben beide den PostgreSQL-Typ `jsonb`.

Ihr Inhalt wird von der Methode `fetchall` automatisch in Objekte einer geeigneten Python-Klasse umgewandelt.

In [39]:
users = cur.fetchall()
users

[('Gates', {'end': 1975, 'start': 1973, 'school_name': 'Harvard University'})]

In [40]:
type(users[0][0])

str

In [41]:
type(users[0][1])

dict

Für komplexere Operationen (wie bspw. die Abfrage der Organisationen aller Positionen des:der Nutzer:in) reicht die "Eckige-Klammer-Notation" nicht aus.
Stattdessen benötigen wir die PosgreSQL-Methode `jsonb_path_query_array` und ein Pfad-Argument entsprechend dem JSONPath-Standard:

In [42]:
cur.execute("""
    select
        doc['last_name'],
        jsonb_path_query_array(doc, '$.positions[*].organization')
    from users
""")
cur.fetchall()

[('Gates', ['Bill & Melinda Gates Foundation', 'Microsoft'])]

##### Dokumente einschränken

Die "Eckige-Klammer-Notation" können wir auch im `where`-Teil der SQL-Query nutzen. Auf diese Weise können wir zum Beispiel nach Nutzern mit einem bestimmten Nachnamen suchen:

In [43]:
cur.execute("""
    select
        doc['last_name'],
        jsonb_path_query_array(doc, '$.positions[*].organization')
    from users
    where doc['last_name'] = '"Gates"'
""")
cur.fetchall()

[('Gates', ['Bill & Melinda Gates Foundation', 'Microsoft'])]

**Achtung:** Da `doc['last_name']` in PostgreSQL den Typ `jsonb` hat, erwartet der Gleichheitsoperator im `where`-Teil des obigen Statements auf der rechten Seite des Gleichheitszeichens ein valides JSON-Dokument. Wir müssen den Namen Gates daher als JSON-String darstellen (`"Gates"`) und diesen JSON-String in einem PostgreSQL-String verpacken (`'"Gates"'`).

Verzichten wir auf die doppelten Anführungszeichen, beschwert sich die PostgreSQL-Instanz über den invaliden Token rechts des Gleichheitszeichens:

In [44]:
try:
    cur.execute("""
        select
            doc['last_name'],
            jsonb_path_query_array(doc, '$.positions[*].organization')
        from users
        where doc['last_name'] = 'Gates'
    """)
except psycopg.errors.InvalidTextRepresentation as e:
    print(e)

invalid input syntax for type json
LINE 6:         where doc['last_name'] = 'Gates'
                                         ^
DETAIL:  Token "Gates" is invalid.
CONTEXT:  JSON data, line 1: Gates


#### Dokumente verändern

Die "Eckige-Klammer-Notation" kann auch in Update-Queries benutzt werden. Das funktioniert im `where`-Teil und im `set`-Teil der Query:

In [45]:
cur.execute("""
    update users
    set doc['first_name'] = '"William"'
    where doc['_id'] = '251'
""")
cur.execute("""
    select doc['first_name'], doc['last_name']
    from users
    where doc['_id'] = '251'
""")
cur.fetchall()

[('William', 'Gates')]

#### Dokumente löschen

Auch im `where`-Teil einer Delete-Query kann die "Eckige-Klammer-Notation" verwendet werden:

In [46]:
cur.execute("""
    delete from users
    where doc['last_name'] = '"Gates"'
""")
cur.execute("""
    select doc
    from users
    where doc['_id'] = '251'
""")
cur.fetchall()

[]

#### Referenzen zwischen Dokumenten

Zur Normalisierung der Profile der Nutzer:innen legen wir zwei weitere Tabellen `industries` und `regions` an mit jeweils einer Spalte `doc` vom Typ `jsonb`. Um von Primary-Key- und Foreign-Key-Constraints zu profitieren, lagern wir alle Primär- und Fremdschlüssel in eigene Tabellenspalten aus.

Wir legen die Tabelle `industries` an:

In [47]:
cur.execute("drop table if exists industries")
cur.execute("create table industries (id int primary key, doc jsonb)")
cur.executemany(
    "insert into industries (id, doc) values (%s, %s)",
    [
        (43, psycopg.types.json.Jsonb({
            "name": "Financial Services",
            "description": "Banking, etc."})),
        (48, psycopg.types.json.Jsonb({"name": "Construction"})),
        (131, psycopg.types.json.Jsonb({"name": "Philanthropy"}))
    ]
)
cur.execute("select id, doc from industries")
cur.fetchall()

[(43, {'name': 'Financial Services', 'description': 'Banking, etc.'}),
 (48, {'name': 'Construction'}),
 (131, {'name': 'Philanthropy'})]

Wir legen die Tabelle `regions` an:

In [48]:
cur.execute("drop table if exists regions")
cur.execute("create table regions (id text primary key, doc jsonb)")
cur.executemany(
    "insert into regions (id, doc) values (%s, %s)",
    [
        ("us:7", psycopg.types.json.Jsonb(
            {"name": "Greater Boston Area"})),
        ("us:91", psycopg.types.json.Jsonb(
            {"name": "Greater Seattle Area", "state": "Washington"}))
    ]
)
cur.execute("select id, doc from regions")
cur.fetchall()

[('us:7', {'name': 'Greater Boston Area'}),
 ('us:91', {'name': 'Greater Seattle Area', 'state': 'Washington'})]

Wir löschen die Tabelle `users` und legen sie neu an:

In [49]:
cur.execute("drop table if exists users")
cur.execute("""
    create table users (
        id integer primary key,
        industry_id integer references industries(id),
        region_id text references regions(id),
        doc jsonb
    )
""")
cur.execute(
    "insert into users (id, industry_id, region_id, doc) values (%s, %s, %s, %s)",
    (251, 131, "us:91", psycopg.types.json.Jsonb({
        "first_name": "Bill",
        "last_name": "Gates",
        "summary": "Co-chair of the Bill & Melinda Gates... Active blogger.",
        "positions": [
            {"job_title": "Co-chair",
             "organization": "Bill & Melinda Gates Foundation"},
            {"job_title": "Co-founder, Chairman", "organization": "Microsoft"}
        ],
        "education": [
            {"school_name": "Harvard University", "start": 1973, "end": 1975},
            {"school_name": "Lakeside School, Seattle"}
        ],
        "contact_info": {
            "blog": "https://www.gatesnotes.com/",
            "twitter": "https://twitter.com/BillGates"
        }
    }))
)
cur.execute("select id, industry_id, region_id, doc from users")
cur.fetchall()

[(251,
  131,
  'us:91',
  {'summary': 'Co-chair of the Bill & Melinda Gates... Active blogger.',
   'education': [{'end': 1975,
     'start': 1973,
     'school_name': 'Harvard University'},
    {'school_name': 'Lakeside School, Seattle'}],
   'last_name': 'Gates',
   'positions': [{'job_title': 'Co-chair',
     'organization': 'Bill & Melinda Gates Foundation'},
    {'job_title': 'Co-founder, Chairman', 'organization': 'Microsoft'}],
   'first_name': 'Bill',
   'contact_info': {'blog': 'https://www.gatesnotes.com/',
    'twitter': 'https://twitter.com/BillGates'}})]

**Achtung:** Das Löschen der Tabelle `users` ist natürlich mit Datenverlust verbunden. Wenn sie also bereits wertvolle Nutzer:innen-Daten gesammelt haben, sollten Sie sich eine bessere Migrationsstrategie überlegen.

Durch die Kombination von relationalem Datenmodell (Spalten für Primär- und Fremdschlüssel) und dokumentorientiertem Datenmodell (Spalten für Dokumente), haben wir eine teilweise Normalisierung der Nutzer:innen-Daten erreicht und können trotzdem weiterhin alle relevanten Nutzer:innen-Merkmale in einer Abfrage abfragen:

In [50]:
cur.execute("""
    select u.doc['last_name'], r.doc['name']
    from users u join regions r on u.region_id = r.id
    where u.id = 251
""")
cur.fetchall()

[('Gates', 'Greater Seattle Area')]

Mit dem Ende unseres Beispiels schließen wir Cursor und Datenbankverbindung. Die PostgreSQL-Instanz kann dann die entsprechenenden Ressourcen freigeben.

In [51]:
cur.close()
conn.close()

## Zusammenfassung und Ausblick

**Dokumentenorientierte Datenbankmanagementsysteme**, die JSON als logisches Datenmodell nutzen, ermöglichen die flexible Verwaltung von JSON-Dokumenten. Indizes ermöglichen die schnelle Abfrage von JSON-Dokumenten(siehe nächste Vorlesung). Ein Schema kann zum Beispiel mithilfe des JSON-Schema-Standards festgelegt werden, muss aber von der Anwendung selbst enforciert werden.

**Relationale Datenbanksysteme** unterstützen JSON-Dokumente als Erweiterung ihres relationalen Datenmodells.
Seit 2016 ist diese Unterstützung Teil des SQL-Sprachstandards. Geeignete Indizes ermöglichen die schnelle Abfrage von JSON-Dokumenten (siehe nächste Vorlesung).

Das relationale Datenmodell ermöglicht
- die starre Modellierung von Entitäten und Beziehungen (in Form eines relationalen Schemas) und
- die Abfrage von Informationen über merere Entitätentypen und Beziehungstypen hinweg.

Die Erweiterung um (JSON-)Dokumente ermöglicht (sofern das relationale Schema Spalten vom Typ Dokument vorsieht)
- die flexible Erweiterung von Entitäten und Beziehungen um zusätzliche Attribute und
- das flexible Hinzufügen von One-to-Many-Beziehungen zu Entitäten eines bereits vorhandenen Entitätentyps.

### Weiterführende Literatur

- Martin Kleppmann (2015). *Designing Data-Intensive Applications.* Sebastopol, CA: O'Reilly.
- [MongoDB Manual (Version 4.4)](https://www.mongodb.com/docs/v4.4/)
- [PostgreSQL: Documentation: 14](https://www.postgresql.org/docs/14/index.html)